In [1]:
import requests
import time
from datetime import datetime
import pandas as pd
import geopandas as gpd
import folium
from tqdm import tqdm

In [2]:
url = 'https://cinematicket.org/api/v1/movie/movies/?categories=3&city=235:Object&page_size=999'

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
response = requests.get(url, headers=headers).json()

In [3]:
movie_count = len(response['results'])
d = {}
d['id'] = [response['results'][i]['id'] for i in range(movie_count)]
d['name'] = [response['results'][i]['name'] for i in range(movie_count)]
d['movie_rate'] = [response['results'][i]['movie_rate'] for i in range(movie_count)]
d['rates_count'] = [response['results'][i]['rates_count'] for i in range(movie_count)]
movies_df = pd.DataFrame(d)

In [30]:
for i in tqdm(range(movies_df.shape[0])):
    time.sleep(1)
    detail_url = 'https://cinematicket.org/api/v1/cinematicket/home/movies/'
    response_detail = requests.get(detail_url+str(movies_df.loc[i]['id']), headers=headers).json()
    #print(movies_df.loc[i]['id'])
    created_at = response_detail['movie']['created_at']
    movies_df.loc[movies_df['id']==movies_df.loc[i]['id'], 'created_at'] = created_at
    movies_df.loc[movies_df['id']==movies_df.loc[i]['id'], 'cinema_address'] = response_detail['cinemas'][0]['cinemas'][0]['cinema']['address']
    movies_df.loc[movies_df['id']==movies_df.loc[i]['id'], 'cinema_name_fa'] = response_detail['cinemas'][0]['cinemas'][0]['cinema']['name_fa']

100%|██████████| 38/38 [00:52<00:00,  1.39s/it]


In [31]:
a = movies_df.copy()

In [35]:
def get_cinemas_location():
    static_cinema_points = {
        'سینما حافظ تهران' :                    'POINT (51.42616268140596, 35.69254602974092)',
        'سینما دهکده المپیک تهران' :           'POINT (51.26397804703257, 35.76151901644865)',
        'سینما صحرا تهران' :                    'POINT (51.43454901237794, 35.707912892771326)',
        'سینما صحرا تهران' :                    'POINT (51.43452755470618, 35.707904180924736)',
        'سینما جوان تهران' :                    'POINT (51.43628641238085, 35.78549376021636)',
        'سینما تئاتر گلریز' :                   'POINT (51.40609599703529, 35.733149254730705)',
        'سینما قدس تهران' :                     'POINT (51.40658396819857, 35.71129464828836)',
        'سالن سوده' :                            'POINT (51.33683942772283, 35.746943539521574)',
        'سینما میلاد تهران' :                    'POINT (51.447510470048854, 35.6893792640906)',
        'پردیس سینمایی صبا مال' :               'POINT (51.39311596819754, 35.684372475577405)',
        'سینما ایران تهران' :                   'POINT (51.44012879268799, 35.71623747234535)',
        'پردیس سینمایی نارسیس' :                'POINT (51.53558865470763, 35.74582981132521)',
        'پردیس سینمایی پرده طلایی' :             'POINT (51.34338595470332, 35.63574446606407)',
        'سالن تئاتر دماوند' :                   'POINT (51.45684391052673, 35.70502778909879)',
        'تماشاخانه سبحان.' :                    'POINT (51.4615254547084, 35.768029157020024)',
        'سالن همایش\u200cهای امام علی' :        'POINT (51.31425628661024, 35.746620906041265)',
        'پردیس سینمایی هروی سنتر' :             'POINT (51.47907749703658, 35.76680089171342)',
        'سالن نمایش خانه مشق' :                 'POINT (51.46255005958102, 35.59208255349042)',
        'سرای محله زعفرانیه' :                  'POINT (51.408809268202475, 35.81154918389405)',
        'سینما تیراژه ۲' :                      'POINT (51.451366481691046, 35.70897350166831)',
        'سینما سپیده تهران' :                   'POINT (51.39951932587001, 35.70154341021152)',
        'پردیس سینمایی شهرک' :                  'POINT (51.365321012379546, 35.75239731238989)',
        'سالن جام جم همت' :                     'POINT (51.32603288169255, 35.753945929499366)',
        'سینما سروش تهران' :                    'POINT (51.435884641213796, 35.709644787660125)',
        'سالن ایوانک غربی' :                    'POINT (51.3445377700516, 35.7564111036055)',
        'سالن همایش و نمایش طهران' :            'POINT (51.32798945470792, 35.75326684502728)',
        'پردیس سینمایی لوتوس مال' :             'POINT (51.4050380565549, 35.63938137873617)',
        'پردیس تئاتر و موسیقی باغ کتاب' :       'POINT (51.43262425470769, 35.75107087243446)',
        'پردیس سینمایی زندگی' :                 'POINT (51.32641091422993, 35.73369329856013)',
        'سینما دزاشیب تهران' :                  'POINT (51.450480124022924, 35.81015328454983)',
        'پردیس سینمایی مگامال' :                'POINT (51.30797795470598, 35.70512447664897)',
        'پردیس سینمایی معین مال' :              'POINT (51.34768841237744, 35.69757453628004)',
        'سینما ماندانا تهران' :                 'POINT (51.491220412378205, 35.71481149460921)',
        'پردیس سینمایی کورش' :                  'POINT (51.3137274277225, 35.73865169351137)',
        'شهر کتاب نپتون' :                                      'POINT (51.45955589674738 35.70580239900479)',
        'سینما آتامان':'POINT (51.19956405913101 35.502033990376084)',
        'سینما هلال شاهرود': 'POINT (54.9762940544417 36.413961089164985)',
        'پردیس سینمایی پرندمال':'POINT (50.96062681208089 35.47990334349421)',
        'مجموعه فرهنگی روح الله':'POINT (51.47621179674707 35.69962560669695)',
        'سرای محله زرگنده': 'POINT (51.430507939079035 35.77093228176906)',
        
        'تئاتر سینما ایران تهران' :'POINT (51.43973754602489, 35.71580724808212)',
        'سینما اریکه ایرانیان':'POINT (51.35781964278399, 35.77203488062446)',
        'سرای محله سعادت آباد':'POINT (51.37571967957004, 35.786743946924695)',
        'مرکز همایش حیدری (مشق)' :                 'POINT (51.46255005958102, 35.59208255349042)',
        'پردیس سینما گالری ملت':'POINT (51.40755591210088, 35.77698386228651)'
    }
    cinema_points_df = pd.DataFrame.from_dict(static_cinema_points, orient='index').reset_index().rename({'index':'cinema_name_fa', 0:'location'}, axis=1)
    
    return cinema_points_df

In [36]:
movies_df['created_at'] = pd.to_datetime(movies_df['created_at'])
movies_df['satisfied_rate_count'] = (movies_df['rates_count'] * movies_df['movie_rate'] / 5).astype('int64')
movies_df['age'] = (datetime.today().date() - movies_df['created_at'].dt.date)
movies_df['age'] = movies_df['age'].apply(lambda x: x.days)
movies_df['satisfied_rate_per_day'] = round(movies_df['satisfied_rate_count'] / movies_df['age'],1)
movies_df['satisfied_rate_per_day'].fillna(0, inplace=True)
movies_df['movie_rate'] = round(movies_df['movie_rate'],1)
cinema_points_df = get_cinemas_location()
movies_df.drop(['location'], axis=1, inplace=True, errors='ignore')
movies_df = movies_df.merge(cinema_points_df, how='left', on='cinema_name_fa')

In [37]:
movies_df

,id,name,movie_rate,rates_count,created_at,cinema_address,cinema_name_fa,satisfied_rate_count,age,satisfied_rate_per_day,location
0,516,خانه سالمندان بانوان,4.0,937,2020-08-15 22:34:15.565102+00:00,بزرگراه حقانی غرب به شرق، بعد از مترو حقانی، خ...,پردیس تئاتر و موسیقی باغ کتاب,749,1317,0.6,"POINT (51.43262425470769, 35.75107087243446)"
1,6019,کمدی موزیکال هوو,4.4,1697,2023-08-03 06:48:49.921309+00:00,خیابان جمهوری، نرسیده به میدان بهارستان,سینما حافظ تهران,1493,234,6.4,"POINT (51.42616268140596, 35.69254602974092)"
2,6051,اول آشناییمون,4.2,659,2023-09-17 06:19:01.437703+00:00,خیابان شریعتی، بالاتر از پل صدر,سینما جوان تهران,553,189,2.9,"POINT (51.43628641238085, 35.78549376021636)"
3,6049,تهران دهه ۷۰,4.0,497,2023-09-13 14:18:44.415816+00:00,خیابان سیدجمال الدین اسدآبادی (یوسف آباد)، نبش...,سینما تئاتر گلریز,397,193,2.1,"POINT (51.40609599703529, 35.733149254730705)"
4,6295,کمدی موزیکال اسپرسو,4.4,934,2023-12-19 12:21:04.220005+00:00,خیابان شریعتی سه راه طالقانی,سینما صحرا تهران,821,96,8.6,"POINT (51.43452755470618, 35.707904180924736)"
5,6085,پارازیت,3.9,486,2023-10-04 12:17:15.414866+00:00,نیاوران، خیابان دزاشیب، خیابان عمار، کوچه مهر،...,سینما دزاشیب تهران,379,172,2.2,"POINT (51.450480124022924, 35.81015328454983)"
6,6083,دریاکنار,4.0,382,2023-10-04 10:17:49.924349+00:00,میدان صادقیه، بلوار کاشانی، بلوار اباذر,پردیس سینمایی زندگی,305,172,1.8,"POINT (51.32641091422993, 35.73369329856013)"
7,6080,فیتیله پیچ,3.9,329,2023-10-02 13:50:33.043117+00:00,همت غرب، بعداز بوستان جوانمردان، دهکده المپیک،...,سینما دهکده المپیک تهران,256,174,1.5,"POINT (51.26397804703257, 35.76151901644865)"
8,6060,گاوچرون,3.9,479,2023-09-23 11:18:07.158455+00:00,شهرک اکباتان، فاز ۲,پردیس سینمایی مگامال,373,183,2.0,"POINT (51.30797795470598, 35.70512447664897)"
9,6070,جاده چالوس,4.3,878,2023-09-27 10:09:50.802533+00:00,میدان ولی عصر,سینما قدس تهران,755,179,4.2,"POINT (51.40658396819857, 35.71129464828836)"


In [38]:
with pd.ExcelWriter('./theatres_v2.xlsx', engine='xlsxwriter') as excel_writer:
    cols = ['id', 'name', 'movie_rate', 'satisfied_rate_count', 'age','satisfied_rate_per_day', 'cinema_address', 'cinema_name_fa', 'rates_count', 'location', 'created_at']
    movies_df[cols].astype({'created_at':'str'}).style.set_properties(**{'text-align': 'center'}).to_excel(excel_writer, sheet_name='theatres', index=False)
    worksheet = excel_writer.sheets['theatres']
    worksheet.add_table(0, 0, movies_df.shape[0], movies_df.shape[1] - 1, {'columns': [{'header': header} for header in cols]})
    worksheet.autofit()

In [39]:
# geodf
from shapely.wkt import loads
location_df = pd.read_excel('./theatres_v2.xlsx')[['name', 'satisfied_rate_count', 'cinema_name_fa', 'location', 'satisfied_rate_per_day']]
location_df['geometry'] = location_df['location'].str.replace(',', '').apply(lambda x: loads(x) if pd.notna(x) else None)
gdfs = gpd.GeoDataFrame(location_df, crs='epsg:4326')
gdfs.head()

,name,satisfied_rate_count,cinema_name_fa,location,satisfied_rate_per_day,geometry
0,خانه سالمندان بانوان,749,پردیس تئاتر و موسیقی باغ کتاب,"POINT (51.43262425470769, 35.75107087243446)",0.6,POINT (51.43262 35.75107)
1,کمدی موزیکال هوو,1493,سینما حافظ تهران,"POINT (51.42616268140596, 35.69254602974092)",6.4,POINT (51.42616 35.69255)
2,اول آشناییمون,553,سینما جوان تهران,"POINT (51.43628641238085, 35.78549376021636)",2.9,POINT (51.43629 35.78549)
3,تهران دهه ۷۰,397,سینما تئاتر گلریز,"POINT (51.40609599703529, 35.733149254730705)",2.1,POINT (51.40610 35.73315)
4,کمدی موزیکال اسپرسو,821,سینما صحرا تهران,"POINT (51.43452755470618, 35.707904180924736)",8.6,POINT (51.43453 35.70790)


In [40]:
gdfs_v = gdfs[gdfs['geometry'] != None]
gdfs[gdfs['geometry'] == None]

,name,satisfied_rate_count,cinema_name_fa,location,satisfied_rate_per_day,geometry


In [41]:
# folium
upper = 2500
lower = 100
gdfs_v['radius'] = gdfs_v['satisfied_rate_count'].apply(lambda x:int(100+(upper-lower)*(x/gdfs_v['satisfied_rate_count'].max())))
my_map = folium.Map(prefer_canvas=True)
folium.GeoJson(
    gdfs_v,
    marker=folium.Circle(fill_color="blue", color="black", weight=1),
    tooltip=folium.GeoJsonTooltip(fields=["name", "satisfied_rate_count", "cinema_name_fa"]),
    popup=folium.GeoJsonPopup(fields=["name", "satisfied_rate_count", "cinema_name_fa"]),
    style_function=lambda x: {"radius": (x['properties']['radius'])},
    highlight_function=lambda x: {"fillOpacity": 0.4}
).add_to(my_map)
my_map.fit_bounds(my_map.get_bounds())
my_map.add_child(folium.map.LayerControl())

In [42]:
# folium
upper = 2500
lower = 100
gdfs_v['ma'] = gdfs_v['satisfied_rate_count']*gdfs_v['satisfied_rate_per_day']
gdfs_v['radius'] = gdfs_v['ma'].apply(lambda x:int(100+(upper-lower)*(x/gdfs_v['ma'].max())))
my_map = folium.Map(prefer_canvas=True)
folium.GeoJson(
    gdfs_v[gdfs_v['name']!='شارلاتان'],
    marker=folium.Circle(fill_color="blue", color="black", weight=1),
    tooltip=folium.GeoJsonTooltip(fields=["name", "satisfied_rate_count", "cinema_name_fa"]),
    popup=folium.GeoJsonPopup(fields=["name", "satisfied_rate_count", "cinema_name_fa"]),
    style_function=lambda x: {"radius": (x['properties']['radius'])},
    highlight_function=lambda x: {"fillOpacity": 0.4}
).add_to(my_map)
my_map.fit_bounds(my_map.get_bounds())
my_map.add_child(folium.map.LayerControl())